# Python &rarr; OKL Sandbox

AST Grammar info can be found in the [ast docs](https://docs.python.org/3/library/ast.html#abstract-grammar)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from typing import List

from occa import okl
from occa.okl import py2okl

In [3]:
def add_vectors(a: [float],
                b: [float],
                ab: [float]) -> None:
    for i in okl.range(len(a)).tile(16):
      ab[i] = a[i] + b[i]

In [4]:
@okl.kernel
def okl_add_vectors(a: [float],
                    b: List[float],
                    ab: List[float]) -> None:
    for i in okl.range(len(a)).tile(16):
      ab[i] = a[i] + b[i]

In [5]:
okl_add_vectors.__name__

'okl_add_vectors'

In [6]:
print(okl_add_vectors.__okl_source__)

@kernel void okl_add_vectors(float *a,
                             float *b,
                             float *ab) {
  for (int i = 0; i < 10; ++i) {
    ab[i] = a[i] + b[i]
  }
}


In [7]:
@okl.kernel
def okl_add_vectors(a: [float],
                    b: List[float],
                    ab: List[float]) -> 2:
    for i in okl.range(len(a)).tile(16):
      ab[i] = a[i] + b[i]

In [15]:
print(okl_add_vectors.__okl_source__)

ValueError: Error: Cannot handle type annotation: 2
1 | def okl_add_vectors(a: [float],
2 |                     b: List[float],
3 |                     ab: List[float]) -> 2:
  |                                         ^
4 |     for i in okl.range(len(a)).tile(16):
5 |       ab[i] = a[i] + b[i]
